In [100]:
'''
import libraries and set print options
'''
import pandas as pd
import csv
pd.set_option('display.max_rows', 1000)

In [101]:
'''
get data from receipt data manual csv. Keep a clean copy for later reference
'''

data = pd.read_csv('../data/receipt_data_manual.csv') ##read csv as a dataframe
full_data = data.copy()
data.head() ## show the top 5

,receipt_name,full_product_title,general_product_title,store,product_database
0,ORGANIC GALA APPLE,Organic gala apples,Apples,Publix,Organic gala apples
1,ACT STRWBY/BLUEBRY,Activia Probiotic Yogurt,Yogurt,Publix,DANNON ACTIVIA FIBER PROBIOTIC LOWFAT YOGURT ...
2,OREO THINS PISTACH,Oreo Thins Pistachio,Oreos,Publix,OREO THINS COOKIES PISTACHIO 1X10.1 OZ
3,OG GW BABY SPINACH,Organic baby spinach,Spinach,Publix,ORGANIC BABY SPINACH
4,PAPAYA MARADOL,Papaya,Papaya,Publix,PAPAYA CHUNKS


In [102]:
'''
clean nulls to blanks. Stip extra spaces
'''
for i in range(len(data.product_database)):
    if (pd.isnull(data.product_database[i])):
        data.product_database[i]=''
    else:
        data.product_database[i] = data.product_database[i].strip()

## RULES:
### - The zeroth index seems to be brand for the most part (few exceptions)
### - When there are 3 items, the first index is the main product with the 2nd index being a descriptor/flavor/type
### - When there are 4 items (only ~4k examples), the 2nd index is the product, the 0th index is still brand, the 3rd index is the descriptor

# Data Cleaning - Process Is Commented

In [103]:
'''
Drop irrelevant columns
'''

if 'no_brand_descriptor_title' in data.columns:
    DROP_COLS = [
        'receipt_name',
        'full_product_title',
        'general_product_title',
        'store',
        'no_brand_descriptor_title', 
        'no_descriptor_title'
    ]
else:
    DROP_COLS = [
        'receipt_name',
        'full_product_title',
        'general_product_title',
        'store'
    ]
data = data.drop(DROP_COLS, axis=1) ## drop unnecessary columns for now
data.head()

,product_database
0,Organic gala apples
1,DANNON ACTIVIA FIBER PROBIOTIC LOWFAT YOGURT S...
2,OREO THINS COOKIES PISTACHIO 1X10.1 OZ
3,ORGANIC BABY SPINACH
4,PAPAYA CHUNKS


In [104]:
'''
Find the number of commas in each title and only use those with less than 5
'''
data['num_commas'] = data.apply(lambda x: len(x.product_database.split(","))-1, axis=1)

# data = data[data['num_commas'] < 4]
for index in range(len(data.product_database)):
    if (data.num_commas[index] >= 4):
        data.product_database[index] = ''

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [105]:
len(data)

444

In [106]:
'''
This will keep brands and separate it from the rest of the titles with a ",". EX: "FRESH & EASY, CANOLA OIL"
'''
def remove_descriptor(x):
    x_ = x.split(",")
    if len(x_) == 2 or len(x_) == 3:
        if 'fl oz' in x_[1]:
            return x_[0].strip()
        else:
            return x_[0].strip() + ", " + x_[1].strip()
    elif len(x_) == 4:
        return x_[0].strip() + ", " + x_[2].strip()
    else:
        return x_[0].strip()

In [107]:
'''
This will keep brands and separate it from the rest of the titles with a ",". EX: "FRESH & EASY, CANOLA OIL"
'''
def remove_brand_descriptor(x):
    x_ = x.split(",")
    
    if len(x_) > 1:
        if 'fl oz' in x_[1]:
            return x_[0].strip()
        else:
            return x_[1].strip()
    elif len(x_) == 4:
        return x_[2].strip()
    else:
        return x_[0].strip()

In [108]:
'''
Apply the 2 methods to obtain two different columns with the cleaning process applied
'''
data['no_brand_descriptor_title'] = data.apply(lambda x: remove_brand_descriptor(x.product_database), axis=1)
data['no_descriptor_title'] = data.apply(lambda x: remove_descriptor(x.product_database), axis=1)

In [109]:
data.tail() ##print dataframe

,product_database,num_commas,no_brand_descriptor_title,no_descriptor_title
439,SEA SCALLOPS,0,SEA SCALLOPS,SEA SCALLOPS
440,PHILADELPHIA CREAM CHEESE-SOFT LIGHT,0,PHILADELPHIA CREAM CHEESE-SOFT LIGHT,PHILADELPHIA CREAM CHEESE-SOFT LIGHT
441,CORN,0,CORN,CORN
442,"NESTLE, GRAND ASSORTED CHOCOLATE",1,GRAND ASSORTED CHOCOLATE,"NESTLE, GRAND ASSORTED CHOCOLATE"
443,PHILADELPHIA CREAM CHEESE-SOFT LIGHT,0,PHILADELPHIA CREAM CHEESE-SOFT LIGHT,PHILADELPHIA CREAM CHEESE-SOFT LIGHT


In [110]:
'''
Get rid of leading and ending spaces, make all one case, in csv file
'''
        
for i in range(len(full_data.product_database)):
    full_data.receipt_name[i] = '' if pd.isnull(full_data.receipt_name[i]) else (full_data.receipt_name[i].upper()).strip()
    full_data.full_product_title[i] = '' if pd.isnull(full_data.full_product_title[i]) else (full_data.full_product_title[i].lower()).strip()
    full_data.general_product_title[i] = '' if pd.isnull(full_data.general_product_title[i]) else (full_data.general_product_title[i].lower()).strip()
    full_data.store[i] = '' if pd.isnull(full_data.store[i]) else (full_data.store[i].lower()).strip()

In [111]:
'''
Create csv rows where there is a product match. Put all unmatched products in a separate file. Don't include duplicates
'''
csvRows = []
no_match_row = []
normal_set = set(["a"])
csvRows.append(['receipt_name', 'full_product_title', 'general_product_title', 'store', 'database', 'no_brand_descriptor_title', 'no_descriptor_title'])
for i in range(len(full_data.product_database)):
    if (full_data.receipt_name[i] not in normal_set):
        normal_set.add(full_data.receipt_name[i])
        if (data.product_database[i] == ''):
            no_match_row.append([full_data.receipt_name[i], full_data.full_product_title[i], full_data.general_product_title[i], full_data.store[i]])
        else:
            csvRows.append([full_data.receipt_name[i], full_data.full_product_title[i], full_data.general_product_title[i], full_data.store[i], data.product_database[i], data.no_brand_descriptor_title[i], data.no_descriptor_title[i]])

In [112]:
'''
Create new, cleaned csv of manual data
'''
csvfile = '../data/cleaned_receipt_data_manual.csv'
with open(csvfile, "w") as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerows(csvRows)

In [113]:
'''
Create csv of manual data with no matches
'''
csvfile = '../data/no_matches_manual_data.csv'
with open(csvfile, "w") as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerows(no_match_row)